<a href="https://colab.research.google.com/github/RouaBenYahia/r-project/blob/master/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
!pip install -q streamlit langchain PyPDF2 faiss-cpu tiktoken huggingface-hub
!pip install -U  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()


In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 868ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
%%writefile Streamlit.py
import streamlit as st
from PyPDF2 import PdfReader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from htmlTemplate import css, bot_template, user_template
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM


def get_pdf_text(pdf_docs):
  text=""
  for pdf in pdf_docs:
    pdf_reader=PdfReader(pdf)
    for page in pdf_reader.pages:
      text+=page.extract_text()
  return text
def get_text_chuncks(text):
  text_splitter=CharacterTextSplitter(separator="\n",chunk_size=1000,chunk_overlap=300,length_function=len)
  chunks=text_splitter.split_text(text)
  return chunks



def get_vectorstore(text_chunks):
    model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
    embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"}
)

    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore):
    tokenizer = AutoTokenizer.from_pretrained("rouabenyahia/FineTuningModel")
    model = AutoModelForCausalLM.from_pretrained("rouabenyahia/FineTuningModel")
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.5,
        do_sample=True
    )

    llm = HuggingFacePipeline(pipeline=pipe)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )

    return conversation_chain



def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)






st.set_page_config(
    page_title="Chat with mutiple PDFs",
    page_icon=":books:",
)

st.write(css,unsafe_allow_html=True)

if "conversation" not in st.session_state:
  st.session_state.conversation=None


if "chat_history" not in st.session_state:
  st.session_state.chat_history=None

st.header("Chat with multiple PDFS : books:")
user_question=st.text_input("Ask a question about your documents")
if user_question:
  handle_userinput(user_question)



with st.sidebar:
  st.subheader("Your documents")
  pdf_docs=st.file_uploader("Upload your PDFs here and click on 'Processs'",accept_multiple_files=True)
  if st.button("Process"):
    with st.spinner("Processing"):
      raw_text=get_pdf_text(pdf_docs)
      #st.write(raw_text)
      text_chunks=get_text_chuncks(raw_text)
      st.write(text_chunks)
      vectorstore = get_vectorstore(text_chunks)
      st.write(vectorstore)
      #el conversation chain tawa
      # on peut faire ça
      st.session_state.conversation=get_conversation_chain(vectorstore)
    #st.session_state.conversation


    #get the pdf text
    #get the text chuncks
    #create the vector store with the embeddings
  #juste el api key mtaa hugging face bech nhothaa


Overwriting Streamlit.py
Overwriting Streamlit.py


In [ ]:

! wget -q -O - ipv4.icanhazip.com



34.63.175.19
34.63.175.19


In [ ]:


! streamlit run Streamlit.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.63.175.19:8501

⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://stupid-apes-heal.loca.lt
2025-07-16 09:25:21.655391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752657921.789448   15798 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752657921.833221   15798 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 09:25:22.000397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable th